In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [38]:
# Read crime data file
f_path = os.path.join("..", "Resources", "geo_crime_df.csv")
geo_crime_df = pd.read_csv(f_path)
geo_crime_df

,City,"Total Crime Per Capita [10,000]","Violent Crime Per Capita [10,000]","Property Crime Per Capita [10,000]"
0,Baytown,441.953174,35.409232,406.543942
1,Friendswood,95.356650,6.371622,88.985028
2,Galveston,453.856637,50.746600,403.110037
3,Houston,554.726790,101.859472,452.867318
4,La Porte,184.057444,21.480662,162.576782
5,League City,196.442149,9.429058,187.013092
6,Missouri City,167.026029,16.989058,150.036971
7,Pasadena,334.806164,42.167938,292.638226
8,Sugar Land,175.690080,10.204585,165.485495
9,Texas City,394.179826,40.049310,354.130516


In [39]:
geo_crime_df["Lat"] = ""
geo_crime_df["Lng"] = ""
geo_crime_df

,City,"Total Crime Per Capita [10,000]","Violent Crime Per Capita [10,000]","Property Crime Per Capita [10,000]",Lat,Lng
0,Baytown,441.953174,35.409232,406.543942,,
1,Friendswood,95.356650,6.371622,88.985028,,
2,Galveston,453.856637,50.746600,403.110037,,
3,Houston,554.726790,101.859472,452.867318,,
4,La Porte,184.057444,21.480662,162.576782,,
5,League City,196.442149,9.429058,187.013092,,
6,Missouri City,167.026029,16.989058,150.036971,,
7,Pasadena,334.806164,42.167938,292.638226,,
8,Sugar Land,175.690080,10.204585,165.485495,,
9,Texas City,394.179826,40.049310,354.130516,,


In [40]:
 
                                  
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in geo_crime_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    state = "TX"
    #print(f"Searching within {city}, {state}...")
    
    # update address key value
    params['address'] = f"{city},{state}"

    # Execute request
    crime_lat_lng = requests.get(base_url, params=params)
    
    # convert response to json
    crime_lat_lng = crime_lat_lng.json()

    # Add resulting lat & lng to their respective columns at the current iteration's row index
    geo_crime_df.loc[index, "Lat"] = crime_lat_lng["results"][0]["geometry"]["location"]["lat"]
    geo_crime_df.loc[index, "Lng"] = crime_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
geo_crime_df


,City,"Total Crime Per Capita [10,000]","Violent Crime Per Capita [10,000]","Property Crime Per Capita [10,000]",Lat,Lng
0,Baytown,441.953174,35.409232,406.543942,29.7355,-94.9774
1,Friendswood,95.356650,6.371622,88.985028,29.5294,-95.201
2,Galveston,453.856637,50.746600,403.110037,29.3013,-94.7977
3,Houston,554.726790,101.859472,452.867318,29.7604,-95.3698
4,La Porte,184.057444,21.480662,162.576782,29.6658,-95.0194
5,League City,196.442149,9.429058,187.013092,29.5075,-95.0949
6,Missouri City,167.026029,16.989058,150.036971,29.6186,-95.5377
7,Pasadena,334.806164,42.167938,292.638226,29.6911,-95.2091
8,Sugar Land,175.690080,10.204585,165.485495,29.5984,-95.6226
9,Texas City,394.179826,40.049310,354.130516,29.3838,-94.9027


In [46]:
gmaps.configure(api_key=g_key)
# create variables to store the data we plan to use on our plot

# data to be used for the plotting coordinates
locations = geo_crime_df[["Lat","Lng"]]

# data to be used to weight the heatmap
rating = geo_crime_df["Total Crime Per Capita [10,000]"]

# create the mapping figure
fig = gmaps.figure()

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating)
heat_layer.max_intensity = 300
heat_layer.point_radius = 25

# add the heatmap layer to the figure
fig.add_layer(heat_layer)

# display the figure
fig



Figure(layout=FigureLayout(height='420px'))